In [ ]:
#!pip install Keras

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import np_utils
from tensorflow.keras import layers
from sklearn.preprocessing import LabelEncoder

# 讀取資料集

In [3]:
import pandas as pd
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', ';')

In [4]:
df.head(5)

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36            20.7      0.045   
1            6.3              0.30         0.34             1.6      0.049   
2            8.1              0.28         0.40             6.9      0.050   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 45.0                 170.0   1.0010  3.00       0.45   
1                 14.0                 132.0   0.9940  3.30       0.49   
2                 30.0                  97.0   0.9951  3.26       0.44   

   alcohol  quality  
0      8.8        6  
1      9.5        6  
2     10.1        6  


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


# 這是類別預測的部分

In [34]:
def classification_preprocessing(df):
    data = df.copy()
    data["fixed acidity"] /= df["fixed acidity"].max()
    data["volatile acidity"] /= df["volatile acidity"].max()
    data["citric acid"] /= df["citric acid"].max()
    data["residual sugar"] /= df["residual sugar"].max()
    data["chlorides"] /= df["chlorides"].max()
    data["free sulfur dioxide"] /= df["free sulfur dioxide"].max()
    data["total sulfur dioxide"] /= df["total sulfur dioxide"].max()
    data["density"] /= df["density"].max()
    data["pH"] /= df["pH"].max()
    data["sulphates"] /= df["sulphates"].max()
    data["alcohol"] /= df["alcohol"].max()
    
    return data

In [44]:
from sklearn.model_selection import train_test_split

data = classification_preprocessing(df)

X = data.copy()
y = X.pop("quality")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [45]:
y_train = np_utils.to_categorical(y_train.values, 11)
y_test = np_utils.to_categorical(y_test.values, 11)

In [47]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [61]:
model = Sequential()

model.add(Dense(200, activation='relu', input_shape=(11,)))
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(25, activation='relu'))
model.add(Dense(11, activation='softmax'))

# compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy',f1_m,precision_m, recall_m])

# fit the model
history = model.fit(X_train, y_train, validation_split=0.2, batch_size=16, epochs=50)

Epoch 1/50
196/196 [==============================] - 2s 3ms/step - loss: 1.6250 - accuracy: 0.3471 - f1_m: 0.0376 - precision_m: 0.0729 - recall_m: 0.0320 - val_loss: 1.3560 - val_accuracy: 0.4987 - val_f1_m: 0.0000e+00 - val_precision_m: 0.0000e+00 - val_recall_m: 0.0000e+00
Epoch 2/50
196/196 [==============================] - 0s 1ms/step - loss: 1.2725 - accuracy: 0.4685 - f1_m: 0.1089 - precision_m: 0.1494 - recall_m: 0.1019 - val_loss: 1.2724 - val_accuracy: 0.4834 - val_f1_m: 0.1060 - val_precision_m: 0.4405 - val_recall_m: 0.0625
Epoch 3/50
196/196 [==============================] - 0s 1ms/step - loss: 1.2190 - accuracy: 0.4772 - f1_m: 0.2119 - precision_m: 0.4224 - recall_m: 0.1587 - val_loss: 1.2430 - val_accuracy: 0.4872 - val_f1_m: 0.2841 - val_precision_m: 0.5716 - val_recall_m: 0.1964
Epoch 4/50
196/196 [==============================] - 0s 1ms/step - loss: 1.1710 - accuracy: 0.5045 - f1_m: 0.3208 - precision_m: 0.5440 - recall_m: 0.2466 - val_loss: 1.2258 - val_accuracy:

Epoch 32/50
196/196 [==============================] - 0s 1ms/step - loss: 1.0321 - accuracy: 0.5556 - f1_m: 0.4993 - precision_m: 0.5876 - recall_m: 0.4405 - val_loss: 1.1137 - val_accuracy: 0.5421 - val_f1_m: 0.4808 - val_precision_m: 0.5698 - val_recall_m: 0.4184
Epoch 33/50
196/196 [==============================] - 0s 1ms/step - loss: 1.0313 - accuracy: 0.5434 - f1_m: 0.4957 - precision_m: 0.5859 - recall_m: 0.4356 - val_loss: 1.1151 - val_accuracy: 0.5446 - val_f1_m: 0.4732 - val_precision_m: 0.5700 - val_recall_m: 0.4082
Epoch 34/50
196/196 [==============================] - 0s 1ms/step - loss: 1.0345 - accuracy: 0.5498 - f1_m: 0.5056 - precision_m: 0.5866 - recall_m: 0.4518 - val_loss: 1.2000 - val_accuracy: 0.5255 - val_f1_m: 0.5135 - val_precision_m: 0.5419 - val_recall_m: 0.4898
Epoch 35/50
196/196 [==============================] - 0s 1ms/step - loss: 1.0412 - accuracy: 0.5481 - f1_m: 0.4844 - precision_m: 0.5644 - recall_m: 0.4298 - val_loss: 1.2359 - val_accuracy: 0.5102 

In [62]:
# evaluate the model
loss, accuracy, f1_score, precision, recall = model.evaluate(X_test, y_test)

31/31 [==============================] - 0s 1ms/step - loss: 1.0239 - accuracy: 0.5694 - f1_m: 0.5277 - precision_m: 0.5943 - recall_m: 0.4764


# 這是數值預測的部分

In [14]:
def regression_preprocessing(df):
    data = df.copy()
    data["fixed acidity"] /= df["fixed acidity"].max()
    data["volatile acidity"] /= df["volatile acidity"].max()
    data["citric acid"] /= df["citric acid"].max()
    data["residual sugar"] /= df["residual sugar"].max()
    data["chlorides"] /= df["chlorides"].max()
    data["free sulfur dioxide"] /= df["free sulfur dioxide"].max()
    data["total sulfur dioxide"] /= df["total sulfur dioxide"].max()
    data["density"] /= df["density"].max()
    data["sulphates"] /= df["sulphates"].max()
    data["alcohol"] /= df["alcohol"].max()
    data.pop("quality")
    
    return data

In [15]:
from keras import backend as K

def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

In [16]:
data = regression_preprocessing(df)

X = data.copy()
y = X.pop("pH")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [17]:
model = Sequential()

model.add(Dense(10, activation='relu', input_shape=(10,)))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1))

model.compile("rmsprop", "mse", metrics=["mae", "mape", rmse])

history = model.fit(X_train, y_train, validation_split=0.2, batch_size=16, epochs=50)

model.fit(X_train, y_train, batch_size=16, epochs=50)

Epoch 1/50
196/196 [==============================] - 1s 2ms/step - loss: 4.8266 - mae: 1.9476 - mape: 61.1220 - rmse: 1.9476 - val_loss: 0.2453 - val_mae: 0.4553 - val_mape: 14.0796 - val_rmse: 0.4553
Epoch 2/50
196/196 [==============================] - 0s 1ms/step - loss: 1.1618 - mae: 0.8769 - mape: 27.4514 - rmse: 0.8769 - val_loss: 0.1188 - val_mae: 0.3040 - val_mape: 9.3386 - val_rmse: 0.3040
Epoch 3/50
196/196 [==============================] - 0s 1ms/step - loss: 0.6133 - mae: 0.6265 - mape: 19.6818 - rmse: 0.6265 - val_loss: 0.0505 - val_mae: 0.1800 - val_mape: 5.4872 - val_rmse: 0.1800
Epoch 4/50
196/196 [==============================] - 0s 1ms/step - loss: 0.4920 - mae: 0.5575 - mape: 17.4359 - rmse: 0.5575 - val_loss: 0.0418 - val_mae: 0.1617 - val_mape: 4.9249 - val_rmse: 0.1617
Epoch 5/50
196/196 [==============================] - 0s 1ms/step - loss: 0.4143 - mae: 0.5176 - mape: 16.2043 - rmse: 0.5176 - val_loss: 0.0243 - val_mae: 0.1196 - val_mape: 3.6889 - val_rmse: 0

Epoch 42/50
196/196 [==============================] - 0s 1ms/step - loss: 0.0191 - mae: 0.1072 - mape: 3.3566 - rmse: 0.1072 - val_loss: 0.0180 - val_mae: 0.1061 - val_mape: 3.3282 - val_rmse: 0.1061
Epoch 43/50
196/196 [==============================] - 0s 1ms/step - loss: 0.0199 - mae: 0.1116 - mape: 3.4873 - rmse: 0.1116 - val_loss: 0.0188 - val_mae: 0.1062 - val_mape: 3.2968 - val_rmse: 0.1062
Epoch 44/50
196/196 [==============================] - 0s 1ms/step - loss: 0.0190 - mae: 0.1059 - mape: 3.3035 - rmse: 0.1059 - val_loss: 0.0180 - val_mae: 0.1049 - val_mape: 3.2743 - val_rmse: 0.1049
Epoch 45/50
196/196 [==============================] - 0s 1ms/step - loss: 0.0192 - mae: 0.1074 - mape: 3.3542 - rmse: 0.1074 - val_loss: 0.0179 - val_mae: 0.1049 - val_mape: 3.2835 - val_rmse: 0.1049
Epoch 46/50
196/196 [==============================] - 0s 1ms/step - loss: 0.0191 - mae: 0.1079 - mape: 3.3729 - rmse: 0.1079 - val_loss: 0.0179 - val_mae: 0.1054 - val_mape: 3.3054 - val_rmse: 0.

In [18]:
# evaluate the model
loss, mae, mape, rmse = model.evaluate(X_test, y_test)

31/31 [==============================] - 0s 1ms/step - loss: 0.0174 - mae: 0.1020 - mape: 3.1743 - rmse: 0.1020
